In [ ]:
import torch
from accelerate import Accelerator, DistributedDataParallelKwargs
import transformers
from tqdm import tqdm

In [ ]:
from bg2vec.arguments import simcse_parser
    
model_args, data_args, training_args, custom_args = simcse_parser.parse_json_file(
        "model_configurations/bggpt-7b-simcse.json"
    )
accelerator = Accelerator()
transformers.set_seed(training_args.seed)
if training_args.gradient_checkpointing:
    training_args.gradient_checkpointing_kwargs = {"use_reentrant": False}

# Loading the data

In [ ]:
from bg2vec.data_util import PairedDataset, load_raw_datasets

In [ ]:
datasets = load_raw_datasets(data_args, model_args)
train_dataset = PairedDataset(datasets['train'])
valid_dataset = PairedDataset(datasets['validation'])
train_examples = [train_dataset[i] 
                  for i in tqdm(range(len(train_dataset)),desc="Loading train examples...",disable=not accelerator.is_main_process)
]
validation_examples = [
    valid_dataset[i]
    for i in tqdm(
        range(len(valid_dataset)),
        desc="Loading train examples...",
        disable=not accelerator.is_main_process,
    )
]

# Loading the model

In [ ]:
from llm2vec import LLM2Vec

model = LLM2Vec.from_pretrained(
    base_model_name_or_path=model_args.model_name_or_path,
    enable_bidirectional=model_args.bidirectional,
    peft_model_name_or_path=model_args.peft_model_name_or_path,
    merge_peft=True,
    pooling_mode=model_args.pooling_mode,
    max_length=model_args.max_seq_length,
    torch_dtype=getattr(torch, model_args.torch_dtype),
    attn_implementation=model_args.attn_implementation,
    attention_dropout=custom_args.simcse_dropout,
    cache_dir="/data/bggpt/"
)
model

In [ ]:
from bg2vec.model import initialize_peft

In [ ]:
# model organization is LLM2VecModel.model -> HF Model, we have to apply PEFT to the inner model
model.model = initialize_peft(
    model.model,
    lora_r=custom_args.lora_r,
    lora_alpha=2 * custom_args.lora_r,
    lora_dropout=custom_args.lora_dropout,
)
model

## Training

In [ ]:
from llm2vec.loss.utils import load_loss
train_loss = load_loss(custom_args.loss_class, scale=custom_args.loss_scale)
train_loss

In [ ]:
from bg2vec.training import SimCSEDefaultCollator
data_collator = SimCSEDefaultCollator(model.tokenize)

In [ ]:
from bg2vec.training import SimCSETrainer, StopTrainingCallback
trainer = SimCSETrainer(
    model=model,
    args=training_args,
    train_dataset=train_examples,
    eval_dataset=validation_examples,
    data_collator=data_collator,
    tokenizer=model.tokenizer,
    loss_function=train_loss,
)

if custom_args.stop_after_n_steps is not None:
    trainer.add_callback(StopTrainingCallback(custom_args.stop_after_n_steps))
trainer.callback_handler.remove_callback(transformers.integrations.integration_utils.WandbCallback)

In [ ]:
trainer.train()